In [5]:
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import random_split
from transformers import AutoTokenizer,  AutoModelForSequenceClassification
from transformers import pipeline
from torch.utils.data import DataLoader
import torch
from torch.optim import Adam

# 1：加载数据,去观察数据的结构（这里有label和review）


In [6]:
import pandas as pd

## 加载数据,去观察数据的结构（这里有label和review）
data = pd.read_csv('D:\\Transformer\\LaLa\\ChnSentiCorp_htl_all.csv')
print(data.head())  # data.head() 是 Pandas 库中用于快速预览 DataFrame 前几行数据的方法。此方法对于初步了解数据结构非常有用。
data = data.dropna()# 去除空数据，清洗数据

   label                                             review
0      1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...
1      1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2      1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3      1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4      1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风


# 2: 创建dataset，用于在训练中进行正式的使用

In [7]:
class MyDataset(Dataset):# 继承,先创建一个数据集对象
    def __init__(self) -> None:# 安全性考量罢了,构造器
        super().__init__()
        self.data = pd.read_csv('D:\\Transformer\\LaLa\\ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna() # 去除空数据

    def __getitem__(self, index): # 观察数据结构，一次返回一行的对应数据,调用索引的时候会自动调用该方法
        return self.data.iloc[index]['review'], self.data.iloc[index]['label']
    def __len__(self) : # 返回数据的数量
        return len(self.data)
dataset = MyDataset() # 这个数据集就创建好了

# 3: 划分数据集
这里拆分的是训练集和验证集


In [8]:
trainset, validset = random_split(dataset,lengths= [0.9 , 0.1]) # 训练集和验证集以9:1的比例进行划分
# len(trainsets)可以观察数据的数量
## 创建dataloader，可以一次返回很多数据
# batch_size 定义每次迭代过程中加载到内存中的样本数量。设置该值为 32 表示每批处理 32 条记录，在一次完整的 epoch 中将遍历所有批次直到覆盖全部数据集。
# 通过设定 shuffle=True 可以使每个 epoch 开始前打乱数据顺序，从而提高模型泛化能力并防止过拟合现象的发生。这对于大多数监督学习任务是有益处的，因为它有助于打破可能存在于原始数据集中任何潜在模式的影响。
# collate_fn 是 PyTorch 中 DataLoader 类的一个重要参数，用于定义如何将一批数据样本组合成一个批次 (batch)1。默认情况下，DataLoader 使用内置的 default_collate 函数来处理这一过程。
tokenizer = AutoTokenizer.from_pretrained("E:\\model\\rbt3")# （我这里tokenizer是下载到本地了，你们想用的时候要参考官网Huggingface关于该模型的说明）,tokenizer是模型的处理文本工具，每个模型都有对应的tokenizer
def collate_func(batch):# 这里用于如何组合一个又一个的batch
    texts, labels = [],[]# 创建两个空列表，一个储存所有文本，一个储存所有标签
    for item in batch:#遍历每一个batch中的每一个元素
        texts.append(item[0])# 上面getitem方法决定了先文本后标签的顺序，因此这里也要按照这样的顺序
        labels.append(item[1])# 给列表添加元素
    # 对输入的数据做一个统一长度规范，最大字数128（切分成一堆128），少则补齐，多则再划分
    # 同时也对inputs本身进行编码,转化为张量形式
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)# 标签转化为张量
    return inputs

  # 这里聚合这些数据就应用了上面你定义的方法
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

# 4: 创建模型以及优化器

In [9]:
model =  AutoModelForSequenceClassification.from_pretrained("E:\\model\\rbt3")# 这个一定要写对，不同的模型用不同的调用方法！血的教训  （我这里模型是下载到本地了，你们想用的时候要参考官网Huggingface关于该模型的说明）
optimizer = Adam(model.parameters(), lr=2e-5)# Adam算法,对于学习率进行优化
if torch.cuda.is_available():
    model = model.cuda()# 能用GPU加载模型就用GPU

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at E:\model\rbt3 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5:训练与验证
## 训练方法

In [10]:
def evaluate(): # 返回预测准确率，这里当然是使用验证集
    model.eval()  #在深度学习框架中，`model.eval()`方法主要用于切换模型到评估模式。
    acc_num = 0
    with torch.inference_mode(): # `torch.inference_mode()` 是 PyTorch 提供的一个上下文管理器，用于启用推理模式，在此模式下可以更高效地执行模型推理。当进入该模式时，自动求导机制会被禁用，从而减少内存占用并提高计算效率

        for batch in validloader:  # 对于验证集的每一个batch
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()} # 让batch加载到GPU上
            output = model(**batch)# 前向传播
            pred = torch.argmax(output.logits, dim=-1) # 使用 PyTorch 的 `torch.argmax` 函数来找到特定张量沿某个维度的最大值索引。要沿着最后一个维度（即类别维度）查找每个样本的最大值索引，可以通过设置 `dim=-1` 参数实现。这就是分类任务中的操作，最大值就是预测概率最大的那个类 （比如有10个类,那就要看这10个类里哪个类预测概率最大，预测结果就是哪个类）
            acc_num += (pred.long() == batch["labels"].long()).float().sum()# 用于计算预测结果与真实标签之间的匹配数量
    return acc_num / len(validset) # 返回预测准确率

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train() # `model.train()` 是一种设置模型进入训练模式的方法
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad() # 梯度清零
            output = model(**batch)  # 前向传播
            output.loss.backward()  # 反向传播
            optimizer.step()  # 参数更新
            if global_step % log_step == 0:# 打印结果
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1  # 遍历所有batch
        acc = evaluate()  # 返回准确率
        print(f"ep: {ep}, acc: {acc}")

## 训练，更新参数，保存模型

In [11]:
train()
tokenizer.save_pretrained("E:\\model\\new rbt3")
model.save_pretrained("E:\\model\\new rbt3") # 保存模型

ep: 0, global_step: 0, loss: 0.6650616526603699
ep: 0, global_step: 100, loss: 0.2918718457221985
ep: 0, global_step: 200, loss: 0.3274252414703369
ep: 0, acc: 0.8853092789649963
ep: 1, global_step: 300, loss: 0.2742455005645752
ep: 1, global_step: 400, loss: 0.22241681814193726
ep: 1, acc: 0.894329845905304
ep: 2, global_step: 500, loss: 0.09457289427518845
ep: 2, global_step: 600, loss: 0.2516028881072998
ep: 2, acc: 0.8994845151901245


# 这下模型就适配你的任务方向了

In [16]:
from transformers import AutoTokenizer,  AutoModelForSequenceClassification
from transformers import pipeline


tokenizer = AutoTokenizer.from_pretrained("E:\\model\\new rbt3")
model =  AutoModelForSequenceClassification.from_pretrained("E:\\model\\new rbt3")# 一定要写对这个，不然就是血的教训
sen = "酒店设施比较差。"
id2_label = {0: "差评！", 1: "好评！"} # 设置分类标签
model.config.id2label = id2_label # 标签映射
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

print(pipe(sen))

Device set to use cuda:0


[{'label': '差评！', 'score': 0.9340776801109314}]
